# 🏗️ 1.4 第一個完整 API

在這個單元中，我們將建立一個具有完整 CRUD 功能的待辦事項 API！

---

## 🎯 學習目標

- ✅ 理解 CRUD 操作 (Create, Read, Update, Delete)
- ✅ 使用 Pydantic 定義資料模型
- ✅ 處理請求體 (Request Body)
- ✅ 回傳適當的 HTTP 狀態碼
- ✅ 建立完整的待辦事項 API

---

## 📋 什麼是 CRUD？

CRUD 是四種基本資料操作的縮寫：

| 操作 | HTTP 方法 | 說明 | 範例 |
|------|-----------|------|------|
| **C**reate | POST | 建立資料 | 新增待辦事項 |
| **R**ead | GET | 讀取資料 | 查看待辦事項 |
| **U**pdate | PUT/PATCH | 更新資料 | 修改待辦事項 |
| **D**elete | DELETE | 刪除資料 | 刪除待辦事項 |

---

## 📦 Pydantic 資料模型

在開始之前，我們需要了解 **Pydantic**，它用於定義和驗證資料結構。

In [1]:
from pydantic import BaseModel
from typing import Optional
from datetime import datetime

# 定義待辦事項的資料模型
class TodoItem(BaseModel):
    """待辦事項模型"""
    id: Optional[int] = None          # ID（可選，由系統生成）
    title: str                         # 標題（必填）
    description: Optional[str] = None  # 描述（可選）
    completed: bool = False            # 是否完成（預設 False）
    created_at: Optional[datetime] = None  # 建立時間

# 測試模型
todo = TodoItem(title="學習 FastAPI")
print(todo)
print(f"JSON 格式: {todo.model_dump_json()}")

id=None title='學習 FastAPI' description=None completed=False created_at=None
JSON 格式: {"id":null,"title":"學習 FastAPI","description":null,"completed":false,"created_at":null}


### Pydantic 的優點

1. **自動驗證** - 資料類型自動檢查
2. **自動轉換** - 自動將字串轉為正確類型
3. **清晰錯誤** - 驗證失敗時給出明確錯誤訊息
4. **JSON 支援** - 輕鬆轉換為 JSON

In [2]:
# 示範 Pydantic 的自動驗證
from pydantic import BaseModel, ValidationError

class User(BaseModel):
    name: str
    age: int

# 正確的資料
user1 = User(name="小明", age=18)
print(f"✅ 正確: {user1}")

# 自動類型轉換
user2 = User(name="小華", age="25")  # 字串 "25" 會自動轉為 int
print(f"✅ 自動轉換: {user2}")

# 錯誤的資料
try:
    user3 = User(name="小美", age="abc")  # 無法轉換為 int
except ValidationError as e:
    print(f"❌ 驗證錯誤: {e}")

✅ 正確: name='小明' age=18
✅ 自動轉換: name='小華' age=25
❌ 驗證錯誤: 1 validation error for User
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='abc', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_parsing


---

## 🚀 建立待辦事項 API

現在讓我們建立完整的待辦事項 API！

In [3]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import Optional, List
from datetime import datetime

# ===== 資料模型 =====

class TodoCreate(BaseModel):
    """建立待辦事項時使用"""
    title: str
    description: Optional[str] = None

class TodoUpdate(BaseModel):
    """更新待辦事項時使用"""
    title: Optional[str] = None
    description: Optional[str] = None
    completed: Optional[bool] = None

class Todo(BaseModel):
    """完整的待辦事項模型"""
    id: int
    title: str
    description: Optional[str] = None
    completed: bool = False
    created_at: datetime

# ===== 建立應用 =====

app = FastAPI(
    title="待辦事項 API",
    description="一個簡單的待辦事項管理 API",
    version="1.0.0"
)

# ===== 模擬資料庫 =====

todos_db: List[Todo] = []
current_id = 0

def get_next_id():
    global current_id
    current_id += 1
    return current_id

In [4]:
# ===== API 端點 =====

# 📖 Read - 取得所有待辦事項
@app.get("/todos", response_model=List[Todo])
def get_todos():
    """取得所有待辦事項"""
    return todos_db

# 📖 Read - 取得單一待辦事項
@app.get("/todos/{todo_id}", response_model=Todo)
def get_todo(todo_id: int):
    """根據 ID 取得待辦事項"""
    for todo in todos_db:
        if todo.id == todo_id:
            return todo
    raise HTTPException(status_code=404, detail="待辦事項不存在")

# ✏️ Create - 建立待辦事項
@app.post("/todos", response_model=Todo, status_code=201)
def create_todo(todo_data: TodoCreate):
    """建立新的待辦事項"""
    new_todo = Todo(
        id=get_next_id(),
        title=todo_data.title,
        description=todo_data.description,
        completed=False,
        created_at=datetime.now()
    )
    todos_db.append(new_todo)
    return new_todo

# 🔄 Update - 更新待辦事項
@app.put("/todos/{todo_id}", response_model=Todo)
def update_todo(todo_id: int, todo_data: TodoUpdate):
    """更新待辦事項"""
    for i, todo in enumerate(todos_db):
        if todo.id == todo_id:
            # 只更新有提供的欄位
            update_data = todo_data.model_dump(exclude_unset=True)
            updated_todo = todo.model_copy(update=update_data)
            todos_db[i] = updated_todo
            return updated_todo
    raise HTTPException(status_code=404, detail="待辦事項不存在")

# 🗑️ Delete - 刪除待辦事項
@app.delete("/todos/{todo_id}")
def delete_todo(todo_id: int):
    """刪除待辦事項"""
    for i, todo in enumerate(todos_db):
        if todo.id == todo_id:
            todos_db.pop(i)
            return {"message": f"待辦事項 {todo_id} 已刪除"}
    raise HTTPException(status_code=404, detail="待辦事項不存在")

---

## 🧪 測試 API

讓我們用 TestClient 來測試我們的 API！

In [5]:
from fastapi.testclient import TestClient

client = TestClient(app)

# 重置資料庫
todos_db.clear()
global current_id
current_id = 0

print("=" * 50)
print("📝 測試：建立待辦事項")
print("=" * 50)

# 建立第一個待辦事項
response = client.post("/todos", json={
    "title": "學習 FastAPI",
    "description": "完成第一章的學習"
})
print(f"狀態碼: {response.status_code}")
print(f"回應: {response.json()}")

📝 測試：建立待辦事項
狀態碼: 201
回應: {'id': 1, 'title': '學習 FastAPI', 'description': '完成第一章的學習', 'completed': False, 'created_at': '2025-12-30T15:55:27.039840'}


In [6]:
print("=" * 50)
print("📝 測試：建立更多待辦事項")
print("=" * 50)

# 建立更多待辦事項
client.post("/todos", json={"title": "練習 Pydantic"})
client.post("/todos", json={"title": "建立 CRUD API"})

# 取得所有待辦事項
response = client.get("/todos")
print(f"目前有 {len(response.json())} 個待辦事項：")
for todo in response.json():
    status = "✅" if todo["completed"] else "⬜"
    print(f"  {status} [{todo['id']}] {todo['title']}")

📝 測試：建立更多待辦事項
目前有 3 個待辦事項：
  ⬜ [1] 學習 FastAPI
  ⬜ [2] 練習 Pydantic
  ⬜ [3] 建立 CRUD API


In [7]:
print("=" * 50)
print("🔄 測試：更新待辦事項")
print("=" * 50)

# 將第一個待辦事項標記為完成
response = client.put("/todos/1", json={"completed": True})
print(f"更新後: {response.json()}")

# 查看更新結果
response = client.get("/todos")
print("\n目前狀態：")
for todo in response.json():
    status = "✅" if todo["completed"] else "⬜"
    print(f"  {status} [{todo['id']}] {todo['title']}")

🔄 測試：更新待辦事項
更新後: {'id': 1, 'title': '學習 FastAPI', 'description': '完成第一章的學習', 'completed': True, 'created_at': '2025-12-30T15:55:27.039840'}

目前狀態：
  ✅ [1] 學習 FastAPI
  ⬜ [2] 練習 Pydantic
  ⬜ [3] 建立 CRUD API


In [8]:
print("=" * 50)
print("🗑️ 測試：刪除待辦事項")
print("=" * 50)

# 刪除第二個待辦事項
response = client.delete("/todos/2")
print(f"刪除結果: {response.json()}")

# 查看刪除後的結果
response = client.get("/todos")
print(f"\n剩餘 {len(response.json())} 個待辦事項：")
for todo in response.json():
    status = "✅" if todo["completed"] else "⬜"
    print(f"  {status} [{todo['id']}] {todo['title']}")

🗑️ 測試：刪除待辦事項
刪除結果: {'message': '待辦事項 2 已刪除'}

剩餘 2 個待辦事項：
  ✅ [1] 學習 FastAPI
  ⬜ [3] 建立 CRUD API


In [9]:
print("=" * 50)
print("❌ 測試：錯誤處理")
print("=" * 50)

# 嘗試取得不存在的待辦事項
response = client.get("/todos/999")
print(f"狀態碼: {response.status_code}")
print(f"錯誤訊息: {response.json()}")

❌ 測試：錯誤處理
狀態碼: 404
錯誤訊息: {'detail': '待辦事項不存在'}


---

## 📊 API 端點總覽

我們建立的 API 包含以下端點：

| 方法 | 路徑 | 說明 | 請求體 | 回應 |
|------|------|------|--------|------|
| GET | `/todos` | 取得所有待辦事項 | - | `List[Todo]` |
| GET | `/todos/{id}` | 取得單一待辦事項 | - | `Todo` |
| POST | `/todos` | 建立待辦事項 | `TodoCreate` | `Todo` |
| PUT | `/todos/{id}` | 更新待辦事項 | `TodoUpdate` | `Todo` |
| DELETE | `/todos/{id}` | 刪除待辦事項 | - | 訊息 |

---

## 🎯 練習題

### 練習：新增「標記全部完成」功能

新增一個端點，可以將所有待辦事項標記為完成。

**需求：**
- 路徑：`PUT /todos/complete-all`
- 功能：將所有待辦事項的 `completed` 設為 `True`
- 回應：`{"message": "已標記 X 個待辦事項為完成"}`

In [ ]:
# 練習：在這裡寫你的程式碼

# @app.put("/todos/complete-all")
# def complete_all_todos():
#     pass  # 你的程式碼

<details>
<summary>📝 點擊查看解答</summary>

```python
@app.put("/todos/complete-all")
def complete_all_todos():
    """將所有待辦事項標記為完成"""
    count = 0
    for i, todo in enumerate(todos_db):
        if not todo.completed:
            updated = todo.model_copy(update={"completed": True})
            todos_db[i] = updated
            count += 1
    return {"message": f"已標記 {count} 個待辦事項為完成"}
```

</details>

---

## 📋 第一章總結

恭喜你完成了第一章！讓我們回顧一下學到的內容：

### 學習成果

| 單元 | 學習內容 |
|------|----------|
| 1.1 | FastAPI 簡介、特點、技術基礎 |
| 1.2 | 環境安裝、uv 套件管理器 |
| 1.3 | Hello World、路由基礎 |
| 1.4 | CRUD API、Pydantic 模型 |

### 關鍵概念

- ✅ `FastAPI()` - 建立應用實例
- ✅ `@app.get/post/put/delete` - 路由裝飾器
- ✅ `BaseModel` - Pydantic 資料模型
- ✅ `HTTPException` - 錯誤處理
- ✅ `response_model` - 回應模型驗證

---

## ⏭️ 下一步

第一章完成！在下一章中，我們將深入學習：

👉 **[第二章：基礎概念](../02_basics/)** 
- Python 類型提示
- Pydantic 進階用法
- 依賴注入系統

---

🎉 **太棒了！你已經可以建立基本的 FastAPI 應用了！**